In [1]:
using CSV

In [2]:
using DataFrameDBs

In [63]:
using Dates

In [88]:
using Statistics

In [15]:
ENV["COLUMNS"] = 1000;

## Создадим таблицу и загрум в нее данные из csv

Т.к. исходный файл большой, то используем CSV.Rows, т.е. импортируем построчно не загружая весь csv в память. Минус - изначально все поля у нас будут иметь тип `Union{Missing, String}`

Источник данных - https://www.kaggle.com/mkechinov/ecommerce-behavior-data-from-multi-category-store

In [138]:
table = create_table("ecommerce", from = CSV.Rows("nov.csv", reuse_row=true), show_progress=true)

Time: 0:08:22.6584 written: 67.5 MRows (134.29 KRows/sec), uncompressed size: 10.09 GB, compressed size: 3.77 GB, compression ratio: 2.68m2mm


DFTable path: ecommerce
10×6 DataFrame


│ Row │ column        │ type                   │ rows       │ uncompressed size │ compressed size │ compression ratio │
│     │ Symbol        │ String                 │ String     │ String            │ String          │ Float64           │
├─────┼───────────────┼────────────────────────┼────────────┼───────────────────┼─────────────────┼───────────────────┤
│ 1   │ event_time    │ Union{Missing, String} │ 67.5 MRows │ 1.7 GB            │ 30.41 MB        │ 57.16             │
│ 2   │ event_type    │ Union{Missing, String} │ 67.5 MRows │ 518.5 MB          │ 28.03 MB        │ 18.5              │
│ 3   │ product_id    │ Union{Missing, String} │ 67.5 MRows │ 735.72 MB         │ 390.22 MB       │ 1.89              │
│ 4   │ category_id   │ Union{Missing, String} │ 67.5 MRows │ 1.45 GB           │ 263.31 MB       │ 5.62              │
│ 5   │ category_code │ Union{Missing, String} │ 67.5 MRows │ 1.2 GB            │ 290.52 MB       │ 4.23              │
│ 6   │ brand         │ Union{Missing, 

Мы уже сжали данные в 2 раза (пока только за счет компрессии lz2). В табличке выше колонки описаны столбцы таблицы в `DataFrameDBs` c типами, занимаемым объемом до сжатия и после сжатия

Можно посмотреть на содержимое таблицы

In [139]:
head(table)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
,String?,String?,String?,String?,String?,String?,String?,String?,String?
1,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
2,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
3,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,missing,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
4,2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
5,2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2
6,2019-11-01 00:00:01 UTC,view,1306894,2053013558920217191,computers.notebook,hp,360.09,520772685,816a59f3-f5ae-4ccd-9b23-82aa8c23d33c
7,2019-11-01 00:00:01 UTC,view,1306421,2053013558920217191,computers.notebook,hp,514.56,514028527,df8184cc-3694-4549-8c8c-6b5171877376
8,2019-11-01 00:00:02 UTC,view,15900065,2053013558190408249,missing,rondell,30.86,518574284,5e6ef132-4d7c-4730-8c7f-85aa4082588f
9,2019-11-01 00:00:02 UTC,view,12708937,2053013553559896355,missing,michelin,72.72,532364121,0a899268-31eb-46de-898d-09b2da950b24


* включим отображение прогресса на все запросы к таблице

In [140]:
turnon_progress!(table);

## Займемся конвертацией данных в правильные типы

## category_id в Int64

* проверим есть ли `missing` в этом поле

In [141]:
sum(ismissing.(table.category_id))

Time: 0:00:22.7751 read: 67.5 MRows (2.96 MRows/sec))0:00:0.2580 read: 458.75 KRows (1.78 MRows/sec)0:00:1.0602 read: 2.42 MRows (2.29 MRows/sec)0:00:2.0825 read: 5.11 MRows (2.45 MRows/sec)0:00:2.4012 read: 6.03 MRows (2.51 MRows/sec)0:00:2.7223 read: 6.95 MRows (2.55 MRows/sec)0:00:3.0458 read: 7.86 MRows (2.58 MRows/sec)0:00:3.3627 read: 8.78 MRows (2.61 MRows/sec)0:00:3.6767 read: 9.7 MRows (2.64 MRows/sec)0:00:3.9910 read: 10.62 MRows (2.66 MRows/sec)0:00:5.3144 read: 14.29 MRows (2.69 MRows/sec)0:00:5.6347 read: 15.2 MRows (2.7 MRows/sec)0:00:5.9658 read: 16.12 MRows (2.7 MRows/sec)0:00:6.3032 read: 17.04 MRows (2.7 MRows/sec)0:00:7.1056 read: 19.33 MRows (2.72 MRows/sec)0:00:7.5669 read: 20.71 MRows (2.74 MRows/sec)0:00:8.9555 read: 24.84 MRows (2.77 MRows/sec)0:00:9.4176 read: 26.21 MRows (2.78 MRows/sec)0:00:9.8796 read: 27.59 MRows (2.79 MRows/sec)0:00:10.3414 read: 28.97 MRows (2.8 MRows/sec)0:00:11.1153 read: 31.26 MRows (2.81 MRows/sec)0:00:11.5597 read: 32.57 MRows (2.82 

0

* переименуем `category_id` в `category_id_raw`

In [142]:
rename_column!(table, :category_id, :category_id_raw)

* создадим колонку с нужным преобразованием

In [143]:
c_id = parse.(Int64, table.category_id_raw)

DFColumn{Int64}

* Вставим эту колонку в нашу таблицу под именемен `category_id` при вставке произойдет материализация данных и запись их на диск

In [145]:
add_column!(table, :category_id, c_id, before=:category_code)

Time: 0:00:0.2289 read: 67.5 MRows (294.87 MRows/sec))
Time: 0:00:0.0723 read: 67.5 MRows (933.77 MRows/sec)
Time: 0:00:0.0748 read: 67.5 MRows (902.5 MRows/sec)
Time: 0:00:20.7611 read: 67.5 MRows (3.25 MRows/sec))0:00:0.3674 read: 1.18 MRows (3.21 MRows/sec)0:00:1.1405 read: 3.54 MRows (3.1 MRows/sec)0:00:1.7194 read: 5.31 MRows (3.09 MRows/sec)0:00:2.3678 read: 7.47 MRows (3.16 MRows/sec)0:00:2.8899 read: 9.24 MRows (3.2 MRows/sec)0:00:3.5914 read: 11.34 MRows (3.16 MRows/sec)0:00:4.2340 read: 13.57 MRows (3.2 MRows/sec)0:00:4.8861 read: 15.66 MRows (3.21 MRows/sec)0:00:5.5472 read: 17.89 MRows (3.23 MRows/sec)0:00:6.3297 read: 20.51 MRows (3.24 MRows/sec)0:00:6.9805 read: 22.74 MRows (3.26 MRows/sec)0:00:7.7385 read: 25.3 MRows (3.27 MRows/sec)0:00:8.3889 read: 27.46 MRows (3.27 MRows/sec)0:00:9.1815 read: 30.08 MRows (3.28 MRows/sec)0:00:9.8388 read: 32.18 MRows (3.27 MRows/sec)0:00:10.6200 read: 34.8 MRows (3.28 MRows/sec)0:00:11.3879 read: 37.09 MRows (3.26 MRows/sec)0:00:11.977

In [146]:
head(table)

Time: 0:00:0.0005 read: 65.54 KRows (130.09 MRows/sec)
Time: 0:00:0.0052 read: 65.54 KRows (12.65 MRows/sec)


,event_time,event_type,product_id,category_id_raw,category_id,category_code,brand,price,user_id,user_session
,String?,String?,String?,String?,Int64,String?,String?,String?,String?,String?
1,2019-11-01 00:00:00 UTC,view,1003461,2053013555631882655,2053013555631882655,electronics.smartphone,xiaomi,489.07,520088904,4d3b30da-a5e4-49df-b1a8-ba5943f1dd33
2,2019-11-01 00:00:00 UTC,view,5000088,2053013566100866035,2053013566100866035,appliances.sewing_machine,janome,293.65,530496790,8e5f4f83-366c-4f70-860e-ca7417414283
3,2019-11-01 00:00:01 UTC,view,17302664,2053013553853497655,2053013553853497655,missing,creed,28.31,561587266,755422e7-9040-477b-9bd2-6a6e8fd97387
4,2019-11-01 00:00:01 UTC,view,3601530,2053013563810775923,2053013563810775923,appliances.kitchen.washer,lg,712.87,518085591,3bfb58cd-7892-48cc-8020-2f17e6de6e7f
5,2019-11-01 00:00:01 UTC,view,1004775,2053013555631882655,2053013555631882655,electronics.smartphone,xiaomi,183.27,558856683,313628f1-68b8-460d-84f6-cec7a8796ef2
6,2019-11-01 00:00:01 UTC,view,1306894,2053013558920217191,2053013558920217191,computers.notebook,hp,360.09,520772685,816a59f3-f5ae-4ccd-9b23-82aa8c23d33c
7,2019-11-01 00:00:01 UTC,view,1306421,2053013558920217191,2053013558920217191,computers.notebook,hp,514.56,514028527,df8184cc-3694-4549-8c8c-6b5171877376
8,2019-11-01 00:00:02 UTC,view,15900065,2053013558190408249,2053013558190408249,missing,rondell,30.86,518574284,5e6ef132-4d7c-4730-8c7f-85aa4082588f
9,2019-11-01 00:00:02 UTC,view,12708937,2053013553559896355,2053013553559896355,missing,michelin,72.72,532364121,0a899268-31eb-46de-898d-09b2da950b24


* удалим `category_id_raw`

In [147]:
drop_column!(table, :category_id_raw)

DFTable path: ecommerce
10×6 DataFrame


│ Row │ column        │ type                   │ rows       │ uncompressed size │ compressed size │ compression ratio │
│     │ Symbol        │ String                 │ String     │ String            │ String          │ Float64           │
├─────┼───────────────┼────────────────────────┼────────────┼───────────────────┼─────────────────┼───────────────────┤
│ 1   │ event_time    │ Union{Missing, String} │ 67.5 MRows │ 1.7 GB            │ 30.41 MB        │ 57.16             │
│ 2   │ event_type    │ Union{Missing, String} │ 67.5 MRows │ 518.5 MB          │ 28.03 MB        │ 18.5              │
│ 3   │ product_id    │ Union{Missing, String} │ 67.5 MRows │ 735.72 MB         │ 390.22 MB       │ 1.89              │
│ 4   │ category_id   │ Int64                  │ 67.5 MRows │ 515.0 MB          │ 184.1 MB        │ 2.8               │
│ 5   │ category_code │ Union{Missing, String} │ 67.5 MRows │ 1.2 GB            │ 290.52 MB       │ 4.23              │
│ 6   │ brand         │ Union{Missing, 

## product_id, user_id и price сделаем по аналогии

In [148]:
rename_column!(table, :product_id, :product_id_raw)
rename_column!(table, :user_id, :user_id_raw)
rename_column!(table, :price, :price_raw);


In [149]:
add_column!(table, :product_id, parse.(Int64, table.product_id_raw), before=:category_id)

Time: 0:00:0.0798 read: 67.5 MRows (845.45 MRows/sec)
Time: 0:00:0.0813 read: 67.5 MRows (830.24 MRows/sec)
Time: 0:00:0.0804 read: 67.5 MRows (839.33 MRows/sec)
Time: 0:00:17.7338 read: 67.5 MRows (3.81 MRows/sec))0:00:1.2558 read: 4.72 MRows (3.76 MRows/sec)0:00:1.9062 read: 7.21 MRows (3.78 MRows/sec)0:00:2.6635 read: 10.09 MRows (3.79 MRows/sec)0:00:3.4088 read: 12.98 MRows (3.81 MRows/sec)0:00:5.6867 read: 20.19 MRows (3.55 MRows/sec)0:00:6.3297 read: 22.81 MRows (3.6 MRows/sec)0:00:6.9746 read: 25.43 MRows (3.65 MRows/sec)0:00:7.7255 read: 28.38 MRows (3.67 MRows/sec)0:00:8.3678 read: 30.87 MRows (3.69 MRows/sec)0:00:9.0181 read: 33.42 MRows (3.71 MRows/sec)0:00:9.8924 read: 36.96 MRows (3.74 MRows/sec)0:00:10.7563 read: 40.37 MRows (3.75 MRows/sec)0:00:11.3003 read: 42.47 MRows (3.76 MRows/sec)0:00:11.9604 read: 45.09 MRows (3.77 MRows/sec)0:00:12.4895 read: 47.19 MRows (3.78 MRows/sec)0:00:13.3673 read: 50.53 MRows (3.78 MRows/sec)0:00:13.8972 read: 52.63 MRows (3.79 MRows/sec)

In [150]:
add_column!(table, :user_id, parse.(Int64, table.user_id_raw), before=:user_session)

Time: 0:00:0.0960 read: 67.5 MRows (702.98 MRows/sec)
Time: 0:00:0.0770 read: 67.5 MRows (876.82 MRows/sec)
Time: 0:00:0.4288 read: 67.5 MRows (157.42 MRows/sec))
Time: 0:00:17.4726 read: 67.5 MRows (3.86 MRows/sec))0:00:0.6323 read: 2.69 MRows (4.25 MRows/sec)0:00:1.7353 read: 7.27 MRows (4.19 MRows/sec)0:00:2.6966 read: 11.27 MRows (4.18 MRows/sec)0:00:3.5608 read: 14.61 MRows (4.1 MRows/sec)0:00:4.2003 read: 17.24 MRows (4.1 MRows/sec)0:00:5.0496 read: 20.71 MRows (4.1 MRows/sec)0:00:6.0426 read: 24.64 MRows (4.08 MRows/sec)0:00:6.7014 read: 27.2 MRows (4.06 MRows/sec)0:00:8.4749 read: 33.95 MRows (4.01 MRows/sec)0:00:8.9028 read: 35.52 MRows (3.99 MRows/sec)0:00:10.0454 read: 39.45 MRows (3.93 MRows/sec)0:00:11.0427 read: 43.19 MRows (3.91 MRows/sec)0:00:11.5536 read: 45.02 MRows (3.9 MRows/sec)0:00:12.3815 read: 47.84 MRows (3.86 MRows/sec)0:00:13.0023 read: 50.07 MRows (3.85 MRows/sec)0:00:13.9107 read: 53.54 MRows (3.85 MRows/sec)0:00:15.6548 read: 60.29 MRows (3.85 MRows/sec)0:

In [151]:
add_column!(table, :price, parse.(Float64, table.price_raw), before=:user_id)

Time: 0:00:0.2257 read: 67.5 MRows (299.07 MRows/sec))
Time: 0:00:0.0772 read: 67.5 MRows (873.95 MRows/sec)
Time: 0:00:0.4271 read: 67.5 MRows (158.03 MRows/sec))
Time: 0:00:15.3469 read: 67.5 MRows (4.4 MRows/sec)m)0:00:0.8671 read: 3.87 MRows (4.46 MRows/sec)0:00:1.2916 read: 5.64 MRows (4.36 MRows/sec)0:00:1.9300 read: 8.59 MRows (4.45 MRows/sec)0:00:2.9188 read: 12.98 MRows (4.45 MRows/sec)0:00:3.6552 read: 16.19 MRows (4.43 MRows/sec)0:00:4.4045 read: 19.66 MRows (4.46 MRows/sec)0:00:5.0300 read: 22.48 MRows (4.47 MRows/sec)0:00:6.1326 read: 27.39 MRows (4.47 MRows/sec)0:00:6.7678 read: 30.21 MRows (4.46 MRows/sec)0:00:7.4002 read: 32.9 MRows (4.45 MRows/sec)0:00:8.1459 read: 36.18 MRows (4.44 MRows/sec)0:00:9.0269 read: 40.04 MRows (4.44 MRows/sec)0:00:9.5544 read: 42.47 MRows (4.44 MRows/sec)0:00:10.1779 read: 45.35 MRows (4.46 MRows/sec)0:00:11.0337 read: 48.96 MRows (4.44 MRows/sec)0:00:12.0169 read: 53.08 MRows (4.42 MRows/sec)0:00:12.6504 read: 55.77 MRows (4.41 MRows/sec)0

In [152]:
drop_column!(table, :product_id_raw)
drop_column!(table, :user_id_raw)
drop_column!(table, :price_raw);

## event_time в DateTime

In [153]:
rename_column!(table, :event_time, :event_time_raw)

* посмотрим на то в каком оно формате в csv

In [154]:
materialize(table.event_time_raw[1:10])

Time: 0:00:0.0005 read: 65.54 KRows (126.22 MRows/sec)


10-element Vector{Union{Missing, String}}:
 "2019-11-01 00:00:00 UTC"
 "2019-11-01 00:00:00 UTC"
 "2019-11-01 00:00:01 UTC"
 "2019-11-01 00:00:01 UTC"
 "2019-11-01 00:00:01 UTC"
 "2019-11-01 00:00:01 UTC"
 "2019-11-01 00:00:01 UTC"
 "2019-11-01 00:00:02 UTC"
 "2019-11-01 00:00:02 UTC"
 "2019-11-01 00:00:02 UTC"

* напишим функцию конвертации

In [155]:
date_convert(s) = DateTime(parse.(Int64, SubString.(s, (1:4, 6:7, 9:10, 12:13, 15:16, 18:19)))...)
s = "2019-11-01 00:00:00 UTC"
println(s, "->", date_convert(s))

2019-11-01 00:00:00 UTC->2019-11-01T00:00:00


In [156]:
tmp = date_convert.(table.event_time_raw)

DFColumn{DateTime}

In [157]:
add_column!(table, :event_time, tmp, before = :event_type)

Time: 0:00:0.0805 read: 67.5 MRows (838.52 MRows/sec)
Time: 0:00:0.0679 read: 67.5 MRows (993.41 MRows/sec)
Time: 0:00:0.0714 read: 67.5 MRows (944.79 MRows/sec)
Time: 0:01:2.9321 read: 67.5 MRows (1.07 MRows/sec)))0:00:0.3094 read: 393.22 KRows (1.27 MRows/sec)0:00:0.6891 read: 786.43 KRows (1.14 MRows/sec)0:00:1.0504 read: 1.18 MRows (1.12 MRows/sec)0:00:1.4041 read: 1.57 MRows (1.12 MRows/sec)0:00:1.7908 read: 1.97 MRows (1.1 MRows/sec)0:00:2.2145 read: 2.36 MRows (1.07 MRows/sec)0:00:2.5553 read: 2.75 MRows (1.08 MRows/sec)0:00:2.9033 read: 3.15 MRows (1.08 MRows/sec)0:00:3.2496 read: 3.54 MRows (1.09 MRows/sec)0:00:3.5942 read: 3.93 MRows (1.09 MRows/sec)0:00:3.9753 read: 4.33 MRows (1.09 MRows/sec)0:00:4.3257 read: 4.72 MRows (1.09 MRows/sec)0:00:4.6956 read: 5.11 MRows (1.09 MRows/sec)0:00:5.0951 read: 5.51 MRows (1.08 MRows/sec)0:00:5.4973 read: 5.9 MRows (1.07 MRows/sec)0:00:6.0884 read: 6.49 MRows (1.07 MRows/sec)0:00:6.4406 read: 6.88 MRows (1.07 MRows/sec)0:00:6.7834 read: 

In [158]:
drop_column!(table, :event_time_raw);

## Заменим missing на пустые строки в оставшихся колонках

In [159]:
string_convert(x) = ismissing(x) ? "" : String(x);

In [160]:
rename_column!(table, :event_type, :event_type_raw)
rename_column!(table, :category_code, :category_code_raw)
rename_column!(table, :brand, :brand_raw)

In [161]:
add_column!(table, :event_type, string_convert.(table.event_type_raw), before=:product_id)

Time: 0:00:0.2280 read: 67.5 MRows (296.1 MRows/sec)m)
Time: 0:00:0.0822 read: 67.5 MRows (821.11 MRows/sec)
Time: 0:00:0.2878 read: 67.5 MRows (234.53 MRows/sec))
Time: 0:00:10.9134 read: 67.5 MRows (6.19 MRows/sec))0:00:5.2129 read: 37.16 MRows (7.13 MRows/sec)0:00:5.9312 read: 41.62 MRows (7.02 MRows/sec)0:00:6.8049 read: 46.47 MRows (6.83 MRows/sec)51.05 MRows (6.58 MRows/sec)0:00:8.5042 read: 54.98 MRows (6.47 MRows/sec)0:00:10.0987 read: 63.44 MRows (6.28 MRows/sec)


In [162]:
add_column!(table, :category_code, string_convert.(table.category_code_raw), before=:brand_raw)

Time: 0:00:0.2659 read: 67.5 MRows (253.86 MRows/sec))
Time: 0:00:0.1382 read: 67.5 MRows (488.45 MRows/sec))
Time: 0:00:0.5870 read: 67.5 MRows (115.0 MRows/sec)m)
Time: 0:00:21.7869 read: 67.5 MRows (3.1 MRows/sec)))0:00:0.4119 read: 1.11 MRows (2.71 MRows/sec)0:00:0.8079 read: 2.42 MRows (3.0 MRows/sec)0:00:3.1442 read: 6.75 MRows (2.15 MRows/sec)0:00:3.9215 read: 9.57 MRows (2.44 MRows/sec)0:00:4.5616 read: 11.93 MRows (2.61 MRows/sec)0:00:5.3696 read: 14.68 MRows (2.73 MRows/sec)0:00:6.0132 read: 17.04 MRows (2.83 MRows/sec)0:00:7.6802 read: 22.35 MRows (2.91 MRows/sec)0:00:8.3291 read: 24.44 MRows (2.93 MRows/sec)0:00:9.7170 read: 29.1 MRows (2.99 MRows/sec)0:00:10.8108 read: 32.18 MRows (2.98 MRows/sec)0:00:11.5670 read: 34.8 MRows (3.01 MRows/sec)0:00:12.6049 read: 38.08 MRows (3.02 MRows/sec)0:00:13.2394 read: 40.24 MRows (3.04 MRows/sec)0:00:13.8893 read: 42.34 MRows (3.05 MRows/sec)0:00:14.4300 read: 43.97 MRows (3.05 MRows/sec)0:00:15.1175 read: 46.2 MRows (3.06 MRows/sec)0

In [163]:
add_column!(table, :brand, string_convert.(table.brand_raw), before=:price)

Time: 0:00:0.1047 read: 67.5 MRows (644.95 MRows/sec))
Time: 0:00:0.1177 read: 67.5 MRows (573.71 MRows/sec))
Time: 0:00:1.9799 read: 67.5 MRows (34.09 MRows/sec))
Time: 0:00:20.6199 read: 67.5 MRows (3.27 MRows/sec))0:00:0.6240 read: 2.1 MRows (3.36 MRows/sec)0:00:1.9242 read: 6.49 MRows (3.37 MRows/sec)0:00:2.5733 read: 8.72 MRows (3.39 MRows/sec)0:00:3.9546 read: 13.43 MRows (3.4 MRows/sec)0:00:4.4967 read: 15.07 MRows (3.35 MRows/sec)0:00:6.2935 read: 20.51 MRows (3.26 MRows/sec)0:00:6.9542 read: 22.61 MRows (3.25 MRows/sec)0:00:8.1169 read: 26.35 MRows (3.25 MRows/sec)0:00:11.0661 read: 35.13 MRows (3.17 MRows/sec)0:00:11.9967 read: 38.08 MRows (3.17 MRows/sec)0:00:14.8217 read: 46.53 MRows (3.14 MRows/sec)3.15 MRows/sec)0:00:16.2470 read: 51.38 MRows (3.16 MRows/sec)0:00:16.7768 read: 53.15 MRows (3.17 MRows/sec)0:00:17.2835 read: 54.79 MRows (3.17 MRows/sec)0:00:17.9421 read: 57.21 MRows (3.19 MRows/sec)0:00:18.5999 read: 59.64 MRows (3.21 MRows/sec)0:00:19.3654 read: 62.59 MRow

In [164]:
drop_column!(table, :event_type_raw)
drop_column!(table, :category_code_raw)
drop_column!(table, :brand_raw);

## user_session в UUID

In [165]:
using UUIDs

### Определим кастомные тип колоноки для UUID

In [166]:
DataFrameDBs.ColumnTypes.serialize(::Type{UUID}) = DataFrameDBs.ColumnTypes.Ast(Symbol("UUID"))
DataFrameDBs.ColumnTypes.deserialize(::Val{Symbol("UUID")}) = UUIDs.UUID

In [167]:
rename_column!(table, :user_session, :user_session_raw)

In [168]:
to_uuid(s) = ismissing(s) ? UUID(0) : UUID(s)

to_uuid (generic function with 1 method)

In [129]:
uuid = UUID.(table.user_session_raw)

DFColumn{UUID}

In [169]:
add_column!(table, :user_session, to_uuid.(table.user_session_raw), before=:user_session_raw)

Time: 0:00:0.0496 read: 67.5 MRows (1.36 BRows/sec)
Time: 0:00:0.0486 read: 67.5 MRows (1.39 BRows/sec)
Time: 0:00:0.3786 read: 67.5 MRows (178.28 MRows/sec))
Time: 0:00:26.9523 read: 67.5 MRows (2.5 MRows/sec)m)0:00:0.6499 read: 1.64 MRows (2.52 MRows/sec)2.6 MRows/sec)0:00:2.2916 read: 6.03 MRows (2.63 MRows/sec)0:00:2.8294 read: 7.47 MRows (2.64 MRows/sec)0:00:3.5250 read: 9.31 MRows (2.64 MRows/sec)0:00:4.0955 read: 10.75 MRows (2.62 MRows/sec)0:00:5.0159 read: 13.3 MRows (2.65 MRows/sec)0:00:5.8208 read: 15.53 MRows (2.67 MRows/sec)0:00:6.6555 read: 17.83 MRows (2.68 MRows/sec)0:00:7.8937 read: 21.04 MRows (2.67 MRows/sec)0:00:8.8422 read: 23.53 MRows (2.66 MRows/sec)0:00:9.4080 read: 24.97 MRows (2.65 MRows/sec)0:00:10.1026 read: 26.74 MRows (2.65 MRows/sec)0:00:10.6606 read: 28.18 MRows (2.64 MRows/sec)0:00:12.1582 read: 32.11 MRows (2.64 MRows/sec)0:00:12.8593 read: 33.88 MRows (2.63 MRows/sec)0:00:13.3982 read: 35.26 MRows (2.63 MRows/sec)0:00:13.9533 read: 36.63 MRows (2.63 M

In [172]:
drop_column!(table, :user_session_raw)

DFTable path: ecommerce
10×6 DataFrame


│ Row │ column        │ type     │ rows       │ uncompressed size │ compressed size │ compression ratio │
│     │ Symbol        │ String   │ String     │ String            │ String          │ Float64           │
├─────┼───────────────┼──────────┼────────────┼───────────────────┼─────────────────┼───────────────────┤
│ 1   │ event_time    │ DateTime │ 67.5 MRows │ 515.0 MB          │ 22.05 MB        │ 23.36             │
│ 2   │ event_type    │ String   │ 67.5 MRows │ 518.5 MB          │ 28.03 MB        │ 18.5              │
│ 3   │ product_id    │ Int64    │ 67.5 MRows │ 515.0 MB          │ 249.36 MB       │ 2.07              │
│ 4   │ category_id   │ Int64    │ 67.5 MRows │ 515.0 MB          │ 184.1 MB        │ 2.8               │
│ 5   │ category_code │ String   │ 67.5 MRows │ 1.2 GB            │ 288.77 MB       │ 4.26              │
│ 6   │ brand         │ String   │ 67.5 MRows │ 588.95 MB         │ 258.98 MB       │ 2.27              │
│ 7   │ price         │ Float64  │ 67.5 MRows

## Итогавая таблица

In [173]:
table

DFTable path: ecommerce
10×6 DataFrame


│ Row │ column        │ type     │ rows       │ uncompressed size │ compressed size │ compression ratio │
│     │ Symbol        │ String   │ String     │ String            │ String          │ Float64           │
├─────┼───────────────┼──────────┼────────────┼───────────────────┼─────────────────┼───────────────────┤
│ 1   │ event_time    │ DateTime │ 67.5 MRows │ 515.0 MB          │ 22.05 MB        │ 23.36             │
│ 2   │ event_type    │ String   │ 67.5 MRows │ 518.5 MB          │ 28.03 MB        │ 18.5              │
│ 3   │ product_id    │ Int64    │ 67.5 MRows │ 515.0 MB          │ 249.36 MB       │ 2.07              │
│ 4   │ category_id   │ Int64    │ 67.5 MRows │ 515.0 MB          │ 184.1 MB        │ 2.8               │
│ 5   │ category_code │ String   │ 67.5 MRows │ 1.2 GB            │ 288.77 MB       │ 4.26              │
│ 6   │ brand         │ String   │ 67.5 MRows │ 588.95 MB         │ 258.98 MB       │ 2.27              │
│ 7   │ price         │ Float64  │ 67.5 MRows